In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from scipy import signal

In [ ]:
pickles_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/Pickles/'
A_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/Z/'
B_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/O/'
C_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/N/'
D_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/F/'
E_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/S/'

In [ ]:
with open(pickles_address + 'As_rhyme_features_F.pickle', 'rb') as f:
    As_features_T = pickle.load(f)
with open(pickles_address + 'Bs_rhyme_features_F.pickle', 'rb') as f:
    Bs_features_T = pickle.load(f)
with open(pickles_address + 'Cs_rhyme_features_F.pickle', 'rb') as f:
    Cs_features_T = pickle.load(f)
with open(pickles_address + 'Ds_rhyme_features_F.pickle', 'rb') as f:
    Ds_features_T = pickle.load(f)
with open(pickles_address + 'Es_rhyme_features_F.pickle', 'rb') as f:
    Es_features_T = pickle.load(f)
print('As_features_T', As_features_T.shape)
print('Bs_features_T', Bs_features_T.shape)
print('Cs_features_T', Cs_features_T.shape)
print('Ds_features_T', Ds_features_T.shape)
print('Es_features_T', Es_features_T.shape)

As_features_T (30, 100)
Bs_features_T (30, 100)
Cs_features_T (30, 100)
Ds_features_T (30, 100)
Es_features_T (30, 100)


In [ ]:
X = list(As_features_T.T) + list(Bs_features_T.T) + list(Cs_features_T.T) + list(Ds_features_T.T)
y = [0] * (len(As_features_T.T) + len(Bs_features_T.T)) + [1] * (len(Cs_features_T.T) + len(Ds_features_T.T))
len(X), len(y)

(400, 400)

In [ ]:
for i in range(len(X)):
    X[i] = [X[i][j] if str(X[i][j]) != '-inf' else 0 for j in range(len(X[i]))]
print(X[0])

[7308.894162727762, 17046585.410098843, 1.2247915984298638, 2.3666290036171445, 4.391422254344696, 70466519.2920548, 5385.146695130319, 8032887.914182351, 0.8681014824382405, 1.0419130696161805, 4.407640557295668, 37032692.842255354, 6228.434503684106, 9413733.472240636, 0.2954878845601577, -0.1798672652422928, 4.4129952921545925, 48207129.838923305, 2815.03394334491, 4262703.8364069285, 1.5292077453105286, 2.833112374916438, 5.813156491180066, 12187119.938590921, 270.1522630114137, 83634.18981283528, 2.618976206261792, 8.598281516345839, 6.435958386530757, 156616.4350230233]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)
len(X_train), len(y_train), len(X_test), len(y_test)

(320, 320, 80, 80)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

array([[29,  4],
       [ 1, 46]])

In [ ]:
correctness = [1 if y_test[i] == y_pred[i] else 0 for i in range(len(y_pred))]
print('Accuracy:', (sum(correctness) / len(correctness)) * 100)

Accuracy: 93.75


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.88      0.92        33
           1       0.92      0.98      0.95        47

    accuracy                           0.94        80
   macro avg       0.94      0.93      0.93        80
weighted avg       0.94      0.94      0.94        80



K-Fold Cross Validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
scores = cross_val_score(knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f' % (np.mean(scores)))

Accuracy: 0.930


LeaveOneOut

In [ ]:
from numpy import mean
from numpy import std
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
cv = LeaveOneOut()
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
scores = cross_val_score(knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f' % (np.mean(scores)))

Accuracy: 0.930


Multi-label Classification

In [ ]:
X = list(As_features_T.T) + list(Bs_features_T.T) + list(Cs_features_T.T) + list(Ds_features_T.T) + list(Es_features_T.T)
y = [0] * (len(As_features_T.T) + len(Bs_features_T.T)) + [1] * (len(Cs_features_T.T) + len(Ds_features_T.T)) + [2] *  len(Es_features_T.T)
print(len(X), len(y))

for i in range(len(X)):
    X[i] = [X[i][j] if str(X[i][j]) != '-inf' else 0 for j in range(len(X[i]))]
print(X[0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)
print(len(X_train), len(y_train), len(X_test), len(y_test))

knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(confusion_matrix(y_test, y_pred))

correctness = [1 if y_test[i] == y_pred[i] else 0 for i in range(len(y_pred))]
print('HoldOutMethod Accuracy:', (sum(correctness) / len(correctness)) * 100)
print(classification_report(y_test, y_pred))

cv = KFold(n_splits=10, random_state=1, shuffle=True)
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
scores = cross_val_score(knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('KFold Accuracy: %.3f' % (np.mean(scores)))

cv = LeaveOneOut()
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
scores = cross_val_score(knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('LeaveOneOut Accuracy: %.3f' % (np.mean(scores)))

500 500
[7308.894162727762, 17046585.410098843, 1.2247915984298638, 2.3666290036171445, 4.391422254344696, 70466519.2920548, 5385.146695130319, 8032887.914182351, 0.8681014824382405, 1.0419130696161805, 4.407640557295668, 37032692.842255354, 6228.434503684106, 9413733.472240636, 0.2954878845601577, -0.1798672652422928, 4.4129952921545925, 48207129.838923305, 2815.03394334491, 4262703.8364069285, 1.5292077453105286, 2.833112374916438, 5.813156491180066, 12187119.938590921, 270.1522630114137, 83634.18981283528, 2.618976206261792, 8.598281516345839, 6.435958386530757, 156616.4350230233]
400 400 100 100
[[31  4  0]
 [ 3 38  1]
 [ 0  1 22]]
HoldOutMethod Accuracy: 91.0
              precision    recall  f1-score   support

           0       0.91      0.89      0.90        35
           1       0.88      0.90      0.89        42
           2       0.96      0.96      0.96        23

    accuracy                           0.91       100
   macro avg       0.92      0.92      0.92       100
w